In [2]:
import cv2
import matplotlib.pyplot as plt
import re
import pandas as pd
from PIL import Image
import io
import os

# Imports the Google Cloud client library
from google.cloud import vision
from google.cloud.vision import types

os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="bin/ROK Project-215279b3c99c.json"

plt.figure(figsize=(12, 10))
plt.style.use('dark_background')

<Figure size 864x720 with 0 Axes>

In [3]:
servers = ['1715']
n = 33
p = re.compile('[\(\)\[\]]')
p1 = re.compile('[,.]')

In [4]:
def detect_text(path):
    """Detects text in the file."""
    from google.cloud import vision
    import io
    client = vision.ImageAnnotatorClient()

    # [START vision_python_migration_text_detection]
    with io.open(path, 'rb') as image_file:
        content = image_file.read()

    image = vision.types.Image(content=content)

    response = client.text_detection(image=image)
    texts = response.text_annotations
    
    vertices = (['({},{})'.format(vertex.x, vertex.y)
                    for vertex in texts[0].bounding_poly.vertices])

    return texts[0].description

In [5]:
def chage_to_number(s_power):
    list_of_numbers = re.findall(r'\d+', s_power)
    return int(''.join(list_of_numbers))

In [10]:
# for server in servers:
for server in servers:
    name = list()
    alliance = list()
    power = list()
    
    for i in range(50):
        n = i+1
        initial = Image.open('./data/server/' + server + '/original/' + str(n) + '.png')
        width = initial.size[0]
        height = initial.size[1]

        croped = initial.crop((width*0.21, height*0.25, width*0.85, height*0.94))
        croped.save('./data/server/' + server + '/croped/' + str(n) + '.png')

        chars = detect_text('./data/server/' + server + '/croped/' + str(n) + '.png')

        tmp_texts = list()
        s = ""
        for c in chars:
            if c != '\n':
                s += c
            else:
                tmp_texts.append(s)
                s = ""

        for k in range(len(tmp_texts)):
            if len(tmp_texts) == 0:
                break

            target = tmp_texts.pop(0)
            if p.search(target):
                name.append(target)

                target = tmp_texts.pop(0)
                if p1.search(target) == None:
                    alliance.append(target)
                    power.append(chage_to_number(tmp_texts.pop(0)))
                else:
                    power.append(chage_to_number(target))
                    alliance.append(tmp_texts.pop(0))

            else:
                name.append(target)
                alliance.append('없음')
                power.append(chage_to_number(tmp_texts.pop(0)))

    df = pd.DataFrame({'name':name, 'alliance':alliance, 'power':power})
    df.to_excel('./data/server/' + server + '/result.xlsx')

# plt.imshow(img_thresh, cmap='gray')